In [1]:
import os
import re
import csv
import codecs
import pandas as pd
from datetime import datetime

In [2]:
def to_date(date_reference):
    """
    Funcao que recebe o ano (YYYY), ano e mes (YYYYmm) ou ano, mes e dia (YYYYmmdd)
    no tipo string e o transforma no tipo date.
    
    Args:
        date_reference(str): Ano, ano e mes ou ano, mes e dia.
            Ex.: '2011' ou '201105' ou '20110526'
    
    Returns:
        date: Data de referencia no formato YYYY-mm-dd.
            Ex.: '2011-05-26'
    """
    
    # Define o ano, mes e dia de referencia do arquivo
    if len(date_reference) == 4:
        year = date_reference[:4]
        month = '01'
        day = '01'
    elif len(date_reference) == 6:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = '01'
    elif len(date_reference) == 8:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = date_reference[6:]
    
    # Define a data de referencia do arquivo
    date = pd.to_datetime(year+'-'+month+'-'+day)
    
    return date

In [3]:
def file_references(data_file):
    """
    Funcao que recebe o nome do arquivo original e extrai o assunto e
    data de referencia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201101_GastosDiretos.csv'
    
    Returns:
        date: Data de referencia do arquivo.
            Ex.: '2011-01-01'
        string: Assunto de referencia do arquivo.
            Ex.: 'GastosDiretos'
    """
    # Expressao Regular para:
    # Identificar a data de referencia no nome do arquivo
    date_reference_re = re.compile(r'^[0-9]+')
    # Identificar o assunto no nome do arquivo
    file_subject_re = re.compile(r'_([\w]+)\.')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, data_file).group()
        
    # Define a data de referencia do arquivo
    date = to_date(date_reference)
    
    # Define o assunto do arquivo
    subject = re.search(file_subject_re, data_file).group(1)
    
    return date, subject
    

In [4]:
def clean_string(text):
    """
    Funcao que recebe um texto (string) e o padroniza.
    Os passos são:
    1. Insere o termo SIGILOSO nos registros protegidos por sigilo.
    2. Insere o termo INDISPONIVEL nos registros com detalhamento nao disponivel.
    3. Remove espacos brancos extras no final dos registros.
    
    Args:
        text(str): Texto de um registro do arquivo original.
            Ex.: 'Texto com espacos extras    '
    
    Returns:
        str: Texto padronizado.
            Ex.: 'Texto com espacos extras'
    """
    # Expressao regular que identifica campos protegidos por sigilo
    confidential_re = re.compile(r'protegidas por sigilo')
    unavailable_re = re.compile(r'Detalhamento das informa')
    
    if pd.isnull(text):
        return text
    else:
        c = confidential_re.search(text)
        i = unavailable_re.search(text)
        if c:
            return u'Sigiloso'
        elif i:
            return u'Indisponivel'
        else:
            return text.strip()

In [5]:
def clean_float(text):
    """
    Funcao que recebe um campo de valor numerico com duas casas decimais
    no tipo string e o tranforma no tipo float.
    Os passos sao:
    1. Remove qualquer caractere nao numerico;
    2. Transforma no tipo de dado float;
    3. Divide por 100 para separar as casas decimais;
    
    Args:
        text(str): Valor numerico em tipo string.
            Ex.: '1.500,70'

    Returns:
        float: Valor numerico em tipo float.
            Ex.: 1500.70
    """
    # Expressao regular que identifica caracteres não numericos
    only_number_re = re.compile(r'\D')
    
    if pd.isnull(text):
        return text
    else:
        return float(re.sub(only_number_re, '', text))/100

In [6]:
def clean_date(text, date_reference):
    """
    Funcao que recebe um campo de data no tipo string no formato dd/mm/YYYY
    e o tranforma no tipo date. Se nao for um campo data, é retornada a data
    de referencia do arquivo (para casos de campos sigilosos).
    
    Args:
        text(str): Data em tipo string no formato dd/mm/YYYY.
            Ex.: '15/07/2013'
        date_reference(date): Data de referencia do arquivo origial.
            Ex.: '2013-07-01'
    Returns:
        date: Data em tipo date no formato YYYY-mm-dd.
            Ex.: '2013-07-15'
    """
    # Expressao regular que identifica data no padrão dd/mm/YYYY
    only_date_re = re.compile(r'([0-9]{2}\/[0-9]{2}\/[0-9]{4})')
    
    d = only_date_re.search(str(text))
    if d:
        return pd.to_datetime(text, format='%d/%m/%Y')
    else:
        return date_reference

In [7]:
def load_data_frame(in_fields, in_data_path):
    """
    Funcao que recebe o caminho completo de um arquivo de dados ja codificado
    em UTF-8, prepara-o em um data frame e renomeia as colunas.
    
    Args:
        in_fields(array): Lista com os nomes das colunas para o data frame.
            Ex.: ['cd_campo', 'nm_campo', 'dt_campo', 'vl_campo']
        in_data_path(str): Caminho completo de acesso ao arquivo de dados.
            Ex.: '..\\data\\encoded\\201301_GastosDiretos.csv'
        
    Returns:
        dataframe: Data frame do arquivo original com as colunas renomeadas.
    """
    # Le o arquivo original em um dataframe
    df = pd.read_csv(in_data_path
                    ,sep=';'
                    ,quotechar = '\"'
                    #,quoting = csv.QUOTE_NONE
                    ,low_memory = False
                    ,encoding = 'utf-8'
                    ,dtype = 'object')
    
    #Renomeia as colunas
    df.columns = in_fields
    
    return df

In [8]:
def data_frame_to_csv(df, out_data_path):
    """
    Funcao que recebe o caminho completo para escrita do dataframe e o
    transforma em arquivo CSV, codificado em UTF-8 e separado por tab.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        out_data_path(str): Caminho completo para escrita do dataframe em csv.
            Ex.: '..\\data\\padronized\\Favorecidos_GastosDiretos_2013-01-01.csv'
    """
    df.to_csv(path_or_buf = out_data_path
             ,index = False
             ,sep = '\t'
             ,encoding = 'utf-8')

In [9]:
def select_fields(df, fields_list):
    """
    Funcao que recebe um data frame e retorna uma copia com apenas as
    colunas informadas.
    
    Args:
        df(dataframe): Data frame enviado para recorte.
        fields_list(array): Lista de colunas do dataframe para selecao.
        
    Returns:
        dataframe: Copia do dataframe apenas com as colunas informadas.
    """
    return df.loc[:,fields_list].copy()

In [10]:
def add_data_source_fields(df, **kwargs):
    """
    Funcao que adiciona ao final do dataframe uma sequencia de campos
    referentes a fonte dos dados que o originou.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        data_source(str): Nome da fonte do arquivo original.
            Ex.: 'Portal da Transparência'
        data_file(str): Nome do arquivo original.
            Ex,: 'GastosDiretos_201301.csv'
        
    Returns:
        dataframe: Dataframe com os campos referentes a 
            fonte de dados adicionado.
    """
    df['dt_referencia'] = kwargs['date_reference']
    df['nm_fonte_dados'] = kwargs['data_source']
    df['nm_arquivo_dados'] = kwargs['data_file'] 
    
    return df

In [11]:
def standardize_cpgf(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '201001_CPGF.csv'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\201001_CPGF.csv'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\201001_CPGF.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] = ['cd_orgao_superior'
                          ,'nm_orgao_superior'
                          ,'cd_orgao_subordinado'
                          ,'nm_orgao_subordinado'
                          ,'cd_unidade_gestora'
                          ,'nm_unidade_gestora'
                          ,'nr_ano_extrato'
                          ,'nr_mes_extrato'
                          ,'nr_cpf_portador'
                          ,'nm_portador'
                          ,'nr_cpf_cnpj_favorecido'
                          ,'nm_favorecido'
                          ,'tp_transacao'
                          ,'dt_transacao'
                          ,'vl_transacao']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Padroniza os campos de data e numericos
    df['nr_mes_extrato'] = df['nr_mes_extrato'].astype(int).map("{:02}".format)
    df['nr_anomes_extrato'] = df['nr_ano_extrato']+df['nr_mes_extrato'] 
    df['dt_extrato'] = df['nr_anomes_extrato'].apply(to_date)
    
    df['dt_transacao'] = df['dt_transacao'].apply(lambda x: clean_date(x, kwargs['date_reference']))
    df['vl_transacao'] = df['vl_transacao'].apply(clean_float)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = ['cd_orgao_superior'
                           ,'nm_orgao_superior'
                           ,'cd_orgao_subordinado'
                           ,'nm_orgao_subordinado'
                           ,'cd_unidade_gestora'
                           ,'nm_unidade_gestora'
                           ,'dt_extrato'
                           ,'nr_cpf_portador'
                           ,'nm_portador'
                           ,'nr_cpf_cnpj_favorecido'
                           ,'nm_favorecido'
                           ,'tp_transacao'
                           ,'dt_transacao'
                           ,'vl_transacao'
                           ,'dt_referencia'
                           ,'nm_fonte_dados'
                           ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [12]:
def standardize_files():
    """
    Acessa os arquivos da pasta 03-organized, passa pelo processo de padronizacao
    e armazena-os na pasta 04-standardized.
    """

    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Nome da fonte dos dados
    kwargs['data_source'] = u'Portal da Transparência'
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\03-organized'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\04-standardized'
    
    # Lista dos arquivos organizados 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Log: Mensagem de inicio do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos iniciada.')
    
    # Para cada arquivo na lista de arquivos organizados
    for file in kwargs['data_files']:
        
        # Define o nome do arquivo
        kwargs['data_file'] = file

        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Define o caminho completo de armazenamento do arquivo padronizado
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Define data e assunto de referencia do arquivo
        kwargs['date_reference'], kwargs['subject'] = file_references(kwargs['data_file'])

        # Padroniza os arquivos
        standardize_cpgf(**kwargs)       
      
    # Log: Mensagem de finalizacao do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos finalizada.')
    

In [13]:
def main():
    """
    Padroniza os dados de Cartao de Pagamento do Governo Federal (CPGF)
    Origem: Portal da Transparencia
    """
    
    ### Padroniza os arquivos
    standardize_files()

if __name__ == '__main__':
    main()

2019-08-20 07:37:15.162899: Padronizacao dos arquivos iniciada.
2019-08-20 07:37:18.739281: Arquivo 201301_CPGF.csv padronizado.
2019-08-20 07:37:19.365837: Arquivo 201302_CPGF.csv padronizado.
2019-08-20 07:37:21.099644: Arquivo 201303_CPGF.csv padronizado.
2019-08-20 07:37:24.003268: Arquivo 201304_CPGF.csv padronizado.
2019-08-20 07:37:27.649390: Arquivo 201305_CPGF.csv padronizado.
2019-08-20 07:37:30.437389: Arquivo 201306_CPGF.csv padronizado.
2019-08-20 07:37:33.855519: Arquivo 201307_CPGF.csv padronizado.
2019-08-20 07:37:37.330574: Arquivo 201308_CPGF.csv padronizado.
2019-08-20 07:37:40.521738: Arquivo 201309_CPGF.csv padronizado.
2019-08-20 07:37:43.940212: Arquivo 201310_CPGF.csv padronizado.
2019-08-20 07:37:47.344763: Arquivo 201311_CPGF.csv padronizado.
2019-08-20 07:37:50.566280: Arquivo 201312_CPGF.csv padronizado.
2019-08-20 07:37:53.066993: Arquivo 201401_CPGF.csv padronizado.
2019-08-20 07:37:53.562378: Arquivo 201402_CPGF.csv padronizado.
2019-08-20 07:37:55.382499